# Case Study #2 - Pizza Runner


<p align="center">
<img src="https://8weeksqlchallenge.com/images/case-study-designs/2.png" style="border-radius:10%" width="30%" height="30%">

## Problem Statement
---
Danny seriously loves Japanese food so in the beginning of 2021, he decides to embark upon a risky venture and opens up a cute little restaurant that sells his 3 favourite foods: sushi, curry and ramen. 
Danny wants to use the data to answer a few simple questions about his customers.  He plans on using these insights to help him decide whether he should expand the existing customer loyalty program.


Danny has provided you with a sample of his overall customer data due to privacy issues - but he hopes that these examples are enough for you to write fully functioning SQL queries to help him answer his questions!

## Import Modules and Load to Use
---

In [65]:
import pandas as pd
import duckdb
from datetime import datetime, timedelta

In [66]:
con = duckdb.connect(database=':memory:', read_only=False) # -> Tạo DB connection

## Dataset
---
Danny has shared with you 3 key datasets for this case study:
* runner_orders
* customer_orders
* runner
* pizza_names
* pizza_recipes
* pizza_toppings


This is entity relationship diagram
<p align="center">
<img src="../Image/entity_relationship_diagram_case_study_2.png">

## Table 1: runners

The runners table shows ```the registration_date``` for each new runner

In [67]:
runners = '''
CREATE TABLE runners (
  "runner_id" INTEGER,
  "registration_date" DATE
);
INSERT INTO runners
  ("runner_id", "registration_date")
VALUES
  (1, '2021-01-01'),
  (2, '2021-01-03'),
  (3, '2021-01-08'),
  (4, '2021-01-15');
  '''
con.execute(runners).fetch_df()

,Count
0,4


In [68]:
view_runners='''
SELECT * FROM runners
'''
con.execute(view_runners).fetch_df()

,runner_id,registration_date
0,1,2021-01-01
1,2,2021-01-03
2,3,2021-01-08
3,4,2021-01-15


## Table 2: customer_orders
Customer pizza orders are captured in the customer_orders table with 1 row for each individual pizza that is part of the order.

The pizza_id relates to the type of pizza which was ordered whilst the exclusions are the ingredient_id values which should be removed from the pizza and the extras are the ingredient_id values which need to be added to the pizza.

Note that customers can order multiple pizzas in a single order with varying exclusions and extras values even if the pizza is the same type!

The exclusions and extras columns will need to be cleaned up before using them in your queries.

In [69]:
customer_orders='''
CREATE TABLE customer_orders (
  "order_id" INTEGER,
  "customer_id" INTEGER,
  "pizza_id" INTEGER,
  "exclusions" VARCHAR(4),
  "extras" VARCHAR(4),
  "order_time" TIMESTAMP
);

INSERT INTO customer_orders
  ("order_id", "customer_id", "pizza_id", "exclusions", "extras", "order_time")
VALUES
  ('1', '101', '1', '', '', '2020-01-01 18:05:02'),
  ('2', '101', '1', '', '', '2020-01-01 19:00:52'),
  ('3', '102', '1', '', '', '2020-01-02 23:51:23'),
  ('3', '102', '2', '', NULL, '2020-01-02 23:51:23'),
  ('4', '103', '1', '4', '', '2020-01-04 13:23:46'),
  ('4', '103', '1', '4', '', '2020-01-04 13:23:46'),
  ('4', '103', '2', '4', '', '2020-01-04 13:23:46'),
  ('5', '104', '1', 'null', '1', '2020-01-08 21:00:29'),
  ('6', '101', '2', 'null', 'null', '2020-01-08 21:03:13'),
  ('7', '105', '2', 'null', '1', '2020-01-08 21:20:29'),
  ('8', '102', '1', 'null', 'null', '2020-01-09 23:54:33'),
  ('9', '103', '1', '4', '1, 5', '2020-01-10 11:22:59'),
  ('10', '104', '1', 'null', 'null', '2020-01-11 18:34:49'),
  ('10', '104', '1', '2, 6', '1, 4', '2020-01-11 18:34:49');
'''
con.execute(customer_orders).fetch_df()

,Count
0,14


In [70]:
view_customer_orders='''
SELECT * FROM customer_orders
'''
con.execute(view_customer_orders).fetch_df()

,order_id,customer_id,pizza_id,exclusions,extras,order_time
0,1,101,1,,,2020-01-01 18:05:02
1,2,101,1,,,2020-01-01 19:00:52
2,3,102,1,,,2020-01-02 23:51:23
3,3,102,2,,NaN,2020-01-02 23:51:23
4,4,103,1,4,,2020-01-04 13:23:46
5,4,103,1,4,,2020-01-04 13:23:46
6,4,103,2,4,,2020-01-04 13:23:46
7,5,104,1,null,1,2020-01-08 21:00:29
8,6,101,2,null,null,2020-01-08 21:03:13
9,7,105,2,null,1,2020-01-08 21:20:29


## Table 3: runner_orders
After each orders are received through the system - they are assigned to a runner - however not all orders are fully completed and can be cancelled by the restaurant or the customer.

The pickup_time is the timestamp at which the runner arrives at the Pizza Runner headquarters to pick up the freshly cooked pizzas. The distance and duration fields are related to how far and long the runner had to travel to deliver the order to the respective customer.

There are some known data issues with this table so be careful when using this in your queries - make sure to check the data types for each column in the schema SQL!

In [71]:
runner_orders='''
CREATE TABLE runner_orders (
  "order_id" INTEGER,
  "runner_id" INTEGER,
  "pickup_time" VARCHAR(19),
  "distance" VARCHAR(7),
  "duration" VARCHAR(10),
  "cancellation" VARCHAR(23)
);

INSERT INTO runner_orders
  ("order_id", "runner_id", "pickup_time", "distance", "duration", "cancellation")
VALUES
  ('1', '1', '2020-01-01 18:15:34', '20km', '32 minutes', ''),
  ('2', '1', '2020-01-01 19:10:54', '20km', '27 minutes', ''),
  ('3', '1', '2020-01-03 00:12:37', '13.4km', '20 mins', NULL),
  ('4', '2', '2020-01-04 13:53:03', '23.4', '40', NULL),
  ('5', '3', '2020-01-08 21:10:57', '10', '15', NULL),
  ('6', '3', 'null', 'null', 'null', 'Restaurant Cancellation'),
  ('7', '2', '2020-01-08 21:30:45', '25km', '25mins', 'null'),
  ('8', '2', '2020-01-10 00:15:02', '23.4 km', '15 minute', 'null'),
  ('9', '2', 'null', 'null', 'null', 'Customer Cancellation'),
  ('10', '1', '2020-01-11 18:50:20', '10km', '10minutes', 'null');
  '''
con.execute(runner_orders).fetch_df()

,Count
0,10


In [72]:
view_runner_orders='''
SELECT * FROM runner_orders
'''
con.execute(view_runner_orders).fetch_df()

,order_id,runner_id,pickup_time,distance,duration,cancellation
0,1,1,2020-01-01 18:15:34,20km,32 minutes,
1,2,1,2020-01-01 19:10:54,20km,27 minutes,
2,3,1,2020-01-03 00:12:37,13.4km,20 mins,NaN
3,4,2,2020-01-04 13:53:03,23.4,40,NaN
4,5,3,2020-01-08 21:10:57,10,15,NaN
5,6,3,null,null,null,Restaurant Cancellation
6,7,2,2020-01-08 21:30:45,25km,25mins,null
7,8,2,2020-01-10 00:15:02,23.4 km,15 minute,null
8,9,2,null,null,null,Customer Cancellation
9,10,1,2020-01-11 18:50:20,10km,10minutes,null


## Table 4: pizza_names
At the moment - Pizza Runner only has 2 pizzas available the Meat Lovers or Vegetarian!

In [73]:
pizza_names='''
DROP TABLE IF EXISTS pizza_names;
CREATE TABLE pizza_names (
  "pizza_id" INTEGER,
  "pizza_name" TEXT
);
INSERT INTO pizza_names
  ("pizza_id", "pizza_name")
VALUES
  (1, 'Meatlovers'),
  (2, 'Vegetarian');
SELECT * FROM pizza_names
'''
con.execute(pizza_names).fetch_df()

,pizza_id,pizza_name
0,1,Meatlovers
1,2,Vegetarian


## Table 5: pizza_recipes
Each pizza_id has a standard set of toppings which are used as part of the pizza recipe.

In [74]:
pizza_recipes='''
DROP TABLE IF EXISTS pizza_recipes;
CREATE TABLE pizza_recipes (
  "pizza_id" INTEGER,
  "toppings" TEXT
);
INSERT INTO pizza_recipes
  ("pizza_id", "toppings")
VALUES
  (1, '1, 2, 3, 4, 5, 6, 8, 10'),
  (2, '4, 6, 7, 9, 11, 12');
SELECT * FROM pizza_recipes
'''
con.execute(pizza_recipes).fetch_df()

,pizza_id,toppings
0,1,"1, 2, 3, 4, 5, 6, 8, 10"
1,2,"4, 6, 7, 9, 11, 12"


## Table 6: pizza_toppings
This table contains all of the topping_name values with their corresponding topping_id value

In [75]:
pizza_toppings='''
DROP TABLE IF EXISTS pizza_toppings;
CREATE TABLE pizza_toppings (
  "topping_id" INTEGER,
  "topping_name" TEXT
);
INSERT INTO pizza_toppings
  ("topping_id", "topping_name")
VALUES
  (1, 'Bacon'),
  (2, 'BBQ Sauce'),
  (3, 'Beef'),
  (4, 'Cheese'),
  (5, 'Chicken'),
  (6, 'Mushrooms'),
  (7, 'Onions'),
  (8, 'Pepperoni'),
  (9, 'Peppers'),
  (10, 'Salami'),
  (11, 'Tomatoes'),
  (12, 'Tomato Sauce');
SELECT * FROM pizza_toppings
'''
con.execute(pizza_toppings).fetch_df()

,topping_id,topping_name
0,1,Bacon
1,2,BBQ Sauce
2,3,Beef
3,4,Cheese
4,5,Chicken
5,6,Mushrooms
6,7,Onions
7,8,Pepperoni
8,9,Peppers
9,10,Salami


## Data Cleaning and Transformation
----

In [76]:
temp_customer_orders='''
DROP TABLE IF EXISTS customer_orders_temp;
CREATE TEMP TABLE customer_orders_temp AS
SELECT 
  order_id, 
  customer_id, 
  pizza_id, 
  CASE
	  WHEN exclusions LIKE 'null' OR exclusions = '' THEN null
	  ELSE exclusions
	  END AS exclusions,
  CASE
	  WHEN extras LIKE 'null' OR extras = '' THEN null
	  ELSE extras
	  END AS extras,
	order_time
FROM customer_orders;
SELECT * FROM customer_orders_temp
'''
con.execute(temp_customer_orders).fetch_df()

,order_id,customer_id,pizza_id,exclusions,extras,order_time
0,1,101,1,NaN,NaN,2020-01-01 18:05:02
1,2,101,1,NaN,NaN,2020-01-01 19:00:52
2,3,102,1,NaN,NaN,2020-01-02 23:51:23
3,3,102,2,NaN,NaN,2020-01-02 23:51:23
4,4,103,1,4,NaN,2020-01-04 13:23:46
5,4,103,1,4,NaN,2020-01-04 13:23:46
6,4,103,2,4,NaN,2020-01-04 13:23:46
7,5,104,1,NaN,1,2020-01-08 21:00:29
8,6,101,2,NaN,NaN,2020-01-08 21:03:13
9,7,105,2,NaN,1,2020-01-08 21:20:29


In [77]:
runner_orders_temp='''
DROP TABLE IF EXISTS runner_orders_temp;
CREATE TEMP TABLE runner_orders_temp AS (
  SELECT order_id, runner_id,
        CASE WHEN pickup_time = 'null' THEN null ELSE pickup_time END :: timestamp AS pickup_time,
--use NULLIF to handle blank string '' turns NULL if two expressions are equal, otherwise it returns the first expression.--
        NULLIF(REGEXP_REPLACE(distance, '[^0-9.]', '', 'g'), '') :: numeric AS distance,
        NULLIF(REGEXP_REPLACE(duration, '[^0-9.]', '', 'g'), '') :: numeric AS duration,
        CASE WHEN cancellation IN ('null', 'NaN', '') THEN null ELSE cancellation END AS cancellation
  FROM runner_orders
);
SELECT * FROM runner_orders_temp;
'''
con.execute(runner_orders_temp).fetch_df()

,order_id,runner_id,pickup_time,distance,duration,cancellation
0,1,1,2020-01-01 18:15:34,20.0,32.0,NaN
1,2,1,2020-01-01 19:10:54,20.0,27.0,NaN
2,3,1,2020-01-03 00:12:37,13.4,20.0,NaN
3,4,2,2020-01-04 13:53:03,23.4,40.0,NaN
4,5,3,2020-01-08 21:10:57,10.0,15.0,NaN
5,6,3,NaT,NaN,NaN,Restaurant Cancellation
6,7,2,2020-01-08 21:30:45,25.0,25.0,NaN
7,8,2,2020-01-10 00:15:02,23.4,15.0,NaN
8,9,2,NaT,NaN,NaN,Customer Cancellation
9,10,1,2020-01-11 18:50:20,10.0,10.0,NaN


## Case Study Questions
---

## Pizza Metrics

1. How many pizzas were ordered?

In [78]:
partA_1='''
SELECT COUNT(*) AS pizza_order_count
FROM customer_orders_temp;
'''
con.execute(partA_1).fetch_df()

,pizza_order_count
0,14


2. How many unique customer orders were made?

In [79]:
partA_2='''
SELECT COUNT(DISTINCT(customer_id)) AS unique_customer_orders
FROM customer_orders_temp
'''
con.execute(partA_2).fetch_df()

,unique_customer_orders
0,5


3. How many successful orders were delivered by each runner?

In [80]:
partA_3='''
SELECT runner_id, COUNT(order_id) AS order_count
FROM runner_orders_temp 
WHERE cancellation IS NULL
GROUP BY runner_id
'''
con.execute(partA_3).fetch_df()

,runner_id,order_count
0,1,4
1,2,3
2,3,1


4. How many of each type of pizza was delivered?

In [81]:
partA_4='''
SELECT p.pizza_name, COUNT(c.pizza_id) AS pizza_count
FROM customer_orders_temp c
JOIN runner_orders_temp r
ON c.order_id=r.order_id
JOIN pizza_names AS p
ON c.pizza_id = p.pizza_id
WHERE cancellation IS NULL
GROUP BY p.pizza_name
'''
con.execute(partA_4).fetch_df()

,pizza_name,pizza_count
0,Meatlovers,9
1,Vegetarian,3


5. How many Vegetarian and Meatlovers were ordered by each customer?

In [82]:
partA_5='''
SELECT c.customer_id, p.pizza_name, COUNT(c.order_id):: INT AS order_count
FROM customer_orders_temp c
JOIN pizza_names AS p
ON c.pizza_id = p.pizza_id
GROUP BY c.customer_id, p.pizza_name
'''
con.execute(partA_5).fetch_df()

,customer_id,pizza_name,order_count
0,101,Meatlovers,2
1,102,Meatlovers,2
2,102,Vegetarian,1
3,103,Meatlovers,3
4,103,Vegetarian,1
5,104,Meatlovers,3
6,101,Vegetarian,1
7,105,Vegetarian,1


In [83]:
partA_5='''
SELECT customer_id,
    SUM(CASE WHEN pizza_id = 1 THEN 1 ELSE 0 END):: INT AS meat_lovers,
    SUM(CASE WHEN pizza_id = 2 THEN 1 ELSE 0 END):: INT AS vegetarian
FROM customer_orders_temp
GROUP BY customer_id;
'''
con.execute(partA_5).fetch_df()

,customer_id,meat_lovers,vegetarian
0,101,2,1
1,102,2,1
2,103,3,1
3,104,3,0
4,105,0,1


6. What was the maximum number of pizzas delivered in a single order?

In [84]:
partA_6='''
SELECT c.order_id, COUNT(c.pizza_id) AS pizza_count 
FROM customer_orders_temp c
JOIN runner_orders_temp r
ON c.order_id=r.order_id
WHERE r.cancellation IS NULL
GROUP BY c.order_id
ORDER BY pizza_count DESC
LIMIT 1
'''
con.execute(partA_6).fetch_df()

,order_id,pizza_count
0,4,3


7. For each customer, how many delivered pizzas had at least 1 change and how many had no changes?

In [85]:
partA_7='''
SELECT 
  c.customer_id,
  SUM(
    CASE WHEN c.exclusions IS NOT NULL OR c.extras IS NOT NULL THEN 1
    ELSE 0
    END):: INT AS at_least_1_change,
  SUM(
    CASE WHEN c.exclusions IS NULL AND c.extras IS NULL THEN 1 
    ELSE 0
    END):: INT AS no_change
FROM customer_orders_temp c
JOIN runner_orders_temp r
ON c.order_id=r.order_id
WHERE r.cancellation IS NULL
GROUP BY c.customer_id
ORDER BY c.customer_id;
'''
con.execute(partA_7).fetch_df()

,customer_id,at_least_1_change,no_change
0,101,0,2
1,102,0,3
2,103,3,0
3,104,2,1
4,105,1,0


8. How many pizzas were delivered that had both exclusions and extras?

In [86]:
partA_8='''
SELECT COUNT(c.order_id) AS pizza_count
FROM customer_orders_temp c
JOIN runner_orders_temp r
ON c.order_id=r.order_id
WHERE (c.exclusions IS NOT NULL AND c.extras IS NOT NULL) AND r.cancellation IS NULL
'''
con.execute(partA_8).fetch_df()

,pizza_count
0,1


9. What was the total volume of pizzas ordered for each hour of the day?

In [87]:
partA_9='''
SELECT HOUR(order_time)AS hour_order_time, COUNT(order_id) AS pizza_count
FROM customer_orders_temp
GROUP BY HOUR(order_time)
'''
con.execute(partA_9).fetch_df()

,hour_order_time,pizza_count
0,11,1
1,13,3
2,18,3
3,19,1
4,21,3
5,23,3


10. What was the volume of orders for each day of the week?

In [88]:
partA_10='''
SELECT dayname(order_time) AS date_order_time, COUNT(order_id) AS pizza_count
FROM customer_orders_temp
GROUP BY dayname(order_time)
'''
con.execute(partA_10).fetch_df()

,date_order_time,pizza_count
0,Wednesday,5
1,Thursday,3
2,Saturday,5
3,Friday,1


## B. Runner and Customer Experience
1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)

In [89]:
partB_1='''
SELECT WEEK(registration_date) AS week, COUNT(runner_id) AS runner_count
FROM runners
GROUP BY WEEK(registration_date)
'''
con.execute(partB_1).fetch_df()

,week,runner_count
0,1,1
1,2,1
2,53,2


2. What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?

In [90]:
partB_2='''
SELECT runner_id, AVG(duration) 
FROM runner_orders_temp 
WHERE cancellation IS NULL
GROUP by runner_id
'''
con.execute(partB_2).fetch_df()

,runner_id,avg(duration)
0,1,22.250000
1,2,26.666667
2,3,15.000000


3. Is there any relationship between the number of pizzas and how long the order takes to prepare?

In [91]:
partB_3='''
WITH prepare_time_avg AS(
    SELECT c.order_id, 
            COUNT(c.pizza_id) AS pizza_count, 
            AVG(DATE_DIFF('MINUTE', c.order_time, r.pickup_time)) AS time_avg
    FROM runner_orders_temp r
    JOIN customer_orders_temp c
    ON r.order_id=c.order_id
    WHERE r.cancellation IS NULL
    GROUP BY c.order_id
    )
SELECT pizza_count, AVG(time_avg) AS prepare_time_avg
FROM prepare_time_avg
GROUP BY pizza_count
'''
con.execute(partB_3).fetch_df()

,pizza_count,prepare_time_avg
0,1,12.2
1,2,18.5
2,3,30.0


4. What was the average distance travelled for each customer?

In [92]:
partB_4='''
SELECT c.customer_id, ROUND(AVG(r.distance),2) AS distance_avg
FROM runner_orders_temp r
JOIN customer_orders_temp c
ON r.order_id=c.order_id
WHERE r.cancellation IS NULL
GROUP BY c.customer_id
'''
con.execute(partB_4).fetch_df()

,customer_id,distance_avg
0,101,20.00
1,102,16.73
2,103,23.40
3,104,10.00
4,105,25.00


5. What was the difference between the longest and shortest delivery times for all orders?

In [93]:
partB_5='''
SELECT MAX(duration) - MIN(duration) AS difference_time
FROM runner_orders_temp
'''
con.execute(partB_5).fetch_df()

,difference_time
0,30.0


6. What was the average speed for each runner for each delivery and do you notice any trend for these values?

In [94]:
partB_6='''
SELECT runner_id, order_id, ROUND((distance*60)/duration,2) AS speed
FROM runner_orders_temp
WHERE cancellation IS NULL
GROUP BY runner_id, order_id,speed
ORDER BY runner_id
'''
con.execute(partB_6).fetch_df()

,runner_id,order_id,speed
0,1,1,37.50
1,1,2,44.44
2,1,3,40.20
3,1,10,60.00
4,2,4,35.10
5,2,7,60.00
6,2,8,93.60
7,3,5,40.00


7. What is the successful delivery percentage for each runner?

In [95]:
partB_7='''
WITH sucess_table AS(
    SELECT runner_id,
            SUM(
                CASE WHEN  cancellation IS NULL THEN 1
                ELSE 0
                END):: INT AS sucess_order,
            COUNT(order_id) AS total_order
    FROM runner_orders_temp
    GROUP BY runner_id
)
SELECT *, sucess_order*100/total_order AS sucess_percentage
FROM sucess_table
'''
con.execute(partB_7).fetch_df()

,runner_id,sucess_order,total_order,sucess_percentage
0,1,4,4,100
1,2,3,4,75
2,3,1,2,50


In [96]:
partB_7_2='''
SELECT 
  runner_id, 
  ROUND(100 * SUM(
      CASE WHEN distance IS NULL THEN 0 ELSE 1 END) / COUNT(*), 0) AS sucess_percentage
FROM runner_orders_temp
GROUP BY runner_id
'''
con.execute(partB_7_2).fetch_df()

,runner_id,sucess_percentage
0,1,100.0
1,2,75.0
2,3,50.0


## C. Ingredient Optimisation
1. What are the standard ingredients for each pizza?

In [97]:
partC_1='''
WITH pizza_recipes_splited AS (
    SELECT pizza_id,
    CAST(UNNEST(string_to_array(toppings, ', ')) AS INT) AS topping_id
    FROM pizza_recipes
)
SELECT t1.pizza_id, t1.pizza_name, t2.topping_id, t3.topping_name
FROM pizza_names t1
JOIN pizza_recipes_splited t2 
ON t1.pizza_id = t2.pizza_id
JOIN pizza_toppings t3 
ON t2.topping_id = t3.topping_id
ORDER BY t1.pizza_id, t2.topping_id;
'''
con.execute(partC_1).fetch_df()

,pizza_id,pizza_name,topping_id,topping_name
0,1,Meatlovers,1,Bacon
1,1,Meatlovers,2,BBQ Sauce
2,1,Meatlovers,3,Beef
3,1,Meatlovers,4,Cheese
4,1,Meatlovers,5,Chicken
5,1,Meatlovers,6,Mushrooms
6,1,Meatlovers,8,Pepperoni
7,1,Meatlovers,10,Salami
8,2,Vegetarian,4,Cheese
9,2,Vegetarian,6,Mushrooms


2. What was the most commonly added extra?

In [98]:
partC_2='''
WITH extras_information AS (
    SELECT 
        order_id,
        CAST(UNNEST(string_to_array(extras, ',')) AS INT) AS extras
        FROM customer_orders_temp
)
SELECT e.extras, p.topping_name, COUNT(e.extras) AS times_ordered
FROM extras_information e
JOIN pizza_toppings p
ON p.topping_id = e.extras
GROUP BY e.extras, p.topping_name
ORDER BY times_ordered DESC;
'''
con.execute(partC_2).fetch_df()

,extras,topping_name,times_ordered
0,1,Bacon,4
1,5,Chicken,1
2,4,Cheese,1


3. What was the most common exclusion?

In [99]:
partC_3='''
WITH exclusions_information AS (
    SELECT 
        order_id,
        CAST(UNNEST(string_to_array(exclusions, ',')) AS INT) AS exclusions
        FROM customer_orders_temp
)
SELECT e.exclusions, p.topping_name, COUNT(e.exclusions) AS times_ordered
FROM exclusions_information e
JOIN pizza_toppings p
ON p.topping_id = e.exclusions
GROUP BY e.exclusions, p.topping_name
ORDER BY times_ordered DESC;
'''
con.execute(partC_3).fetch_df()

,exclusions,topping_name,times_ordered
0,4,Cheese,4
1,2,BBQ Sauce,1
2,6,Mushrooms,1


4. Generate an order item for each record in the customers_orders table in the format of one of the following:
<ul>
    <span style="color:pink;">
    <li>Meat Lovers</li>
    <li>Meat Lovers - Exclude Beef</li>
    <li>Meat Lovers - Extra Bacon</li>
    <li>Meat Lovers - Exclude Cheese, Bacon - Extra Mushroom, Peppers</li></span>
<ul>

In [169]:
partC_4='''
WITH exclusion_extras AS
(
  SELECT c.order_id, c.pizza_id, p.pizza_name,
        array_slice(c.exclusions, 0,1) AS exclusions_1,
        array_slice(c.exclusions, 3,4) AS exclusions_2,
        array_slice(c.extras, 0,1) AS extras_1,
        array_slice(c.extras, 3,4) AS extras_2
  FROM customer_orders_temp c
  JOIN pizza_names p
  ON c.pizza_id=p.pizza_id
),
sumary_table AS
(
  SELECT e.order_id, e.pizza_id, e.pizza_name,
        t1.topping_name AS exclusions_1,
        t2.topping_name AS exclusions_2,
        t3.topping_name AS extras_1,
        t4.topping_name AS extras_2
  FROM exclusion_extras e
  LEFT JOIN pizza_toppings t1 ON e.exclusions_1= t1.topping_id
  LEFT JOIN pizza_toppings t2 ON e.exclusions_2= t2.topping_id
  LEFT JOIN pizza_toppings t3 ON e.extras_1= t3.topping_id
  LEFT JOIN pizza_toppings t4 ON e.extras_2 = t4.topping_id
  ORDER BY order_id
),
modify_table AS
(
SELECT order_id, pizza_id, pizza_name,
      CASE
        WHEN exclusions_1 IS NULL AND exclusions_2 IS NULL THEN NULL
        WHEN exclusions_1 IS NOT NULL AND exclusions_2 IS NULL THEN exclusions_1
        WHEN exclusions_1 IS NULL AND exclusions_2 IS NOT NULL THEN exclusions_2
        WHEN exclusions_1 IS NOT NULL AND exclusions_2 IS NOT NULL 
            THEN CONCAT(exclusions_1, ', ', exclusions_2)  
      END AS exclusions,
      CASE
        WHEN extras_1 IS NULL AND extras_2 IS NULL THEN NULL
        WHEN extras_1 IS NOT NULL AND extras_2 IS NULL THEN extras_1
        WHEN extras_1 IS NULL AND extras_2 IS NOT NULL THEN exclusions_2
        WHEN extras_1 IS NOT NULL AND extras_2 IS NOT NULL 
            THEN CONCAT(extras_1, ', ', extras_2)
      END AS extras
FROM sumary_table 
ORDER BY order_id
)
SELECT order_id, pizza_id,
      CASE 
        WHEN exclusions IS NULL AND extras IS NULL THEN pizza_name
        WHEN exclusions IS NULL AND extras IS NOT NULL 
            THEN CONCAT(pizza_name,' - Extra: ',extras)
        WHEN exclusions IS NOT NULL AND extras IS NULL 
            THEN CONCAT(pizza_name,' - Exclude: ',exclusions)
        WHEN exclusions IS NOT NULL AND extras IS NOT NULL 
            THEN CONCAT(pizza_name,' - Exclude: ',exclusions,' - Extra: ',extras)
      END AS detail_order
FROM modify_table
ORDER BY order_id
'''
con.execute(partC_4).fetch_df()

,order_id,pizza_id,detail_order
0,1,1,Meatlovers
1,2,1,Meatlovers
2,3,1,Meatlovers
3,3,2,Vegetarian
4,4,1,Meatlovers - Exclude: Cheese
5,4,1,Meatlovers - Exclude: Cheese
6,4,2,Vegetarian - Exclude: Cheese
7,5,1,Meatlovers - Extra: Bacon
8,6,2,Vegetarian
9,7,2,Vegetarian - Extra: Bacon


5. Generate an alphabetically ordered comma separated ingredient list for each pizza order from the customer_orders table and add a 2x in front of any relevant ingredients
<ul>
    <span style="color:pink;">
    <li>
    For example: "Meat Lovers: 2xBacon, Beef, ... , Salami"
    </li>
    </span>
    
</ul>


6. What is the total quantity of each ingredient used in all delivered pizzas sorted by most frequent first?

In [185]:
partC_6='''
SELECT order_id, pizza_id,
    CASE WHEN exclusions IS NOT NULL
    THEN UNNEST(STRING_TO_ARRAY(exclusions, ','))
    ELSE exclusions
    END AS exclusions,
    CASE WHEN extras IS NOT NULL
    THEN UNNEST(STRING_TO_ARRAY(extras, ','))
    ELSE extras
    END AS extras
FROM customer_orders_temp
'''
con.execute(partC_6).fetch_df()

,order_id,pizza_id,exclusions,extras
0,4,1,4,NaN
1,4,1,4,NaN
2,4,2,4,NaN
3,5,1,NaN,1
4,7,2,NaN,1
5,9,1,4,1
6,9,1,NaN,5
7,10,1,2,1
8,10,1,6,4
